# "Анализ базы данных книжного сервиса. SQL-запросы" <a class="tocSkip">

## Цель исследования

<div style="border: ridge black 3px; padding: 25px">

<b>Целью данного исследования является анализ базы данных книжного сервиса (при помощи SQL-запросов) и выявление ключевых характеристик и данных, которые помогут сервису для чтения книг сформулировать  ценностное предложение для нового продукта.</b> 
   
</div>

## Описание исходных данных

Таблица **`books`**. **Содержит данные о книгах:**
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
___
Таблица **`authors`**. **Содержит данные об авторах:**
- `author_id` — идентификатор автора;
- `author` — имя автора.
___
Таблица **`publishers`**. **Содержит данные об издательствах:**
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
___
Таблица **`ratings`**. **Содержит данные о пользовательских оценках книг:**
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
___
Таблица **`reviews`**. **Содержит данные о пользовательских обзорах:**
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.
___

## Импорт библиотек и подключение к базе данных 

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Для удобства создадим функцию `show_result` для показа результатов SQL-запроса.

In [3]:
def show_result(request):
    return pd.io.sql.read_sql(request, con = engine)

### Просмотр содержимого таблиц

Ознакомимся с содержимым таблиц и тобразим первые 5 строк каждой из них с помощью функции `show_result`.

In [4]:
# пишем запросы

books_req = \
'''
SELECT * 
FROM books
LIMIT 5
'''
authors_req = \
'''
SELECT * 
FROM authors
LIMIT 5
'''
publishers_req = \
'''
SELECT * 
FROM publishers
LIMIT 5
'''
ratings_req = \
'''
SELECT * 
FROM ratings
LIMIT 5
'''
reviews_req = \
'''
SELECT * 
FROM reviews
LIMIT 5
'''

In [5]:
# выводим результат

print('Таблица "Books":')
display(show_result(books_req))
print('-------------------------------------------------------------------------------------------')
print('Таблица "Authors":')
display(show_result(authors_req))
print('-------------------------------------------------------------------------------------------')
print('Таблица "Publishers":')
display(show_result(publishers_req))
print('-------------------------------------------------------------------------------------------')
print('Таблица "Ratings":')
display(show_result(ratings_req))
print('-------------------------------------------------------------------------------------------')
print('Таблица "Reviews":')
display(show_result(reviews_req))
print('-------------------------------------------------------------------------------------------')

Таблица "Books":


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


-------------------------------------------------------------------------------------------
Таблица "Authors":


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


-------------------------------------------------------------------------------------------
Таблица "Publishers":


...

-------------------------------------------------------------------------------------------
Таблица "Ratings":


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


-------------------------------------------------------------------------------------------
Таблица "Reviews":


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


-------------------------------------------------------------------------------------------


Проанализируем базу данных книг и определим ключевые показатели и данные:
1. Количество книг, вышедших после 1 Января 2000 года.
2. Количество обзоров и среднюю оценку по каждой книге.
3. Издательство, выпустившее наибольшее число книг  (книги с более, чем 50 стр.).
4. Автора с самой высокой средней оценкой книг (книги с 50 и более оценками).
5. Среднее количество обзоров от пользователей (поставивших более 50 оценок).

Приступим к созданию запросов.

## SQL-запросы

### Количество книг, вышедших после 1 января 2000 года

In [6]:
n_books_after_01_01_2000 = \
'''
SELECT 
    COUNT(book_id) AS n_books_after_01_01_2000
FROM 
    books
WHERE 
    publication_date >= '2000-01-01';
'''

show_result(n_books_after_01_01_2000)

,n_books_after_01_01_2000
0,821


**После 1 Января 2000 г. (включ.) вышла 821 книга.**

<div style="border:solid green 2px; padding: 20px"> <b>Отличная работа</b><br>
Спасибо за отметку о включении 1 января!</div> <br>

### Количество обзоров и средняя оценка по каждой книге

In [7]:
rating_and_reviews = \
'''
SELECT
    books.book_id AS book_id,
    books.title AS book_title,
    COUNT(DISTINCT reviews.review_id) AS cnt_reviews,   
    AVG(ratings.rating) AS avg_rating   
FROM
    books
    INNER JOIN ratings ON ratings.book_id = books.book_id
    INNER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    cnt_reviews DESC
LIMIT 10;
'''
# выведем первые 10 книг по убыванию кол-ва обзоров
show_result(rating_and_reviews)

,book_id,book_title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


**Видим, что более 6 обзоров получила только 1 книга:**

- Twilight (Twilight #1): 7 обзоров и рейтинг не из самых высоких = 3.66 баллов.

**Остальные 9 книг имеют по 6 обзоров.** 

**Средний рейтинг книг в этой десятке колеблется от 3.4 до 4.4 баллов.**

### Издательство, выпустившее наибольшее число книг  (книги с более, чем 50 стр.)

In [8]:
publisher_largest_n_books = \
'''
SELECT                                                 
    publishers.publisher AS publisher_name,
    COUNT(books.book_id) AS cnt_books
FROM
    publishers
    INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50                                        
GROUP BY
    publisher_name
ORDER BY                                                     
    cnt_books DESC
LIMIT 10;
'''
show_result(publisher_largest_n_books)

,publisher_name,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


**Издательство "Penguin Books" выпустило больше всего книг (42 книги).**

На 2 месте издательство "Vintage" и на 3 месте - "Grand Central Publishing".

### Автор с самой высокой средней оценкой книг (книги с 50 и более оценками)

In [9]:
author_highest_avg_rating = \
'''
SELECT                                                    
    SUBQ.author AS author,                                
    AVG(SUBQ.avg_rating) AS avg_rating
FROM (

    SELECT
         authors.author AS author,
         books.book_id AS id,
         AVG(ratings.rating) AS avg_rating,
         COUNT(ratings.rating) AS cnt_rating
    FROM
        authors
    INNER JOIN books ON books.author_id = authors.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        id
    HAVING
        COUNT(ratings.rating) >= 50) AS SUBQ  
        
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 10     
'''
show_result(author_highest_avg_rating)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самую высокую среднюю оценку книг получила J.K. Rowling/Mary GrandPré - 4.28 балла.

С несущественным отставанием следом идут Markus Zusak/Cao Xuân Việt Khương (4.26 балла) и J.R.R. Tolkien (4.25 балла).

### Среднее количество обзоров от пользователей (поставивших более 50 оценок)

In [10]:
avg_n_reviews = \
'''
SELECT
     SUM(SUBQ.cnt_reviews)/COUNT(SUBQ.cnt_reviews) AS avg_reviews

FROM (  

    SELECT 
        reviews.username, 
        COUNT(reviews.username) as cnt_reviews
    FROM 
        reviews
    WHERE 
        reviews.username IN (
            SELECT ratings.username
            FROM 
                ratings
            GROUP BY 
                ratings.username
            HAVING COUNT(ratings.rating_id) > 50)
GROUP BY 
    username) as SUBQ

'''
show_result(avg_n_reviews)

,avg_reviews
0,24.333333


## Общий вывод

<div style="border: ridge black 3px; padding: 25px">

<b>В результате анализа установлено:</b><br>
    
1. Количество книг, вышедших после 1 Января 2000 года: 821 книга.<br>
    
    
2. 1) Из ТОП-10 книг по количеству обзоров 9 книг получили по 6 обзоров, и только 1 книга-лидера получила 7 обзоров: "Twilight (Twilight #1)".<br>
   2) Средний рейтинг книг (из ТОП-10 книг по количеству обзоров колеблется) колеблется от 3.4 до 4.4 баллов.<br>
    
    
3. Издательство, выпустившее наибольшее число книг  (книги с более, чем с 50 стр.): "Penguin Books"(42 книги).<br>
    На 2 месте издательство "Vintage" (31 книга) и на 3 месте - "Grand Central Publishing" (25 книг).<br>
    
    
4. Автор с самой высокой средней оценкой книг (книги с 50 и более оценками): J.K. Rowling/Mary GrandPré - 4.28 балла.<br>
    
    
5. Среднее количество обзоров от пользователей (поставивших более 50 оценок): 24.3.<br>

</div>